In [1]:
import pyspark

In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
#!pip install py4j==0.10.9.5

In [5]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = (SparkSession
        .builder
        .appName('AuthorsAges')
        .getOrCreate())

In [6]:
data_df = spark.createDataFrame([('Brooke',20), ('Denny',31), ('Jules',30),("TD",35),('Brooke',25)], ['name','age'])

avg_df = data_df.groupBy('name').agg(avg('age'))
avg_df.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



# 스키마 정의

## 프로그래밍 스타일로 정의

In [7]:
from pyspark.sql.types import *

schema = StructType([StructField("author", StringType(), False),
                    StructField("title", StringType(), False)])

In [8]:
schema

StructType([StructField('author', StringType(), False), StructField('title', StringType(), False)])

## DDL 사용하여 정의

In [9]:
schema = "author STRING, title STRING, pages INT"

# p.52 예제

In [10]:
from pyspark.sql import SparkSession

# DDL 사용하여 스키마 정의
#schema = "'ID' INT, 'First' STRING, 'Last' STRING, 'Url' STRING, 'Published' STRING, 'HITS' INT, 'Campaigns' ARRAY<STRING>"

schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("First", StringType(), False),
   StructField("Last", StringType(), False),
   StructField("Url", StringType(), False),
   StructField("Published", StringType(), False),
   StructField("Hits", IntegerType(), False),
   StructField("Campaigns", ArrayType(StringType()), False)])

data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
       [2, "Brooke","Wenig","https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3","6/7/2019",7659, ["web", "twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das","https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]
      ]

# 메인 프로그램

#스파크세션 생성
spark = (SparkSession
        .builder
        .appName("Example-3_6")
        .getOrCreate())

blogs_df = spark.createDataFrame(data, schema)
blogs_df.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+



In [11]:
blogs_df.printSchema()

root
 |-- Id: integer (nullable = false)
 |-- First: string (nullable = false)
 |-- Last: string (nullable = false)
 |-- Url: string (nullable = false)
 |-- Published: string (nullable = false)
 |-- Hits: integer (nullable = false)
 |-- Campaigns: array (nullable = false)
 |    |-- element: string (containsNull = true)



# 로우 (p.58~)

In [12]:
from pyspark.sql import Row

In [13]:
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", ["twitter", "LinkedIn"])

In [14]:
blog_row[1]

'Reynold'

In [15]:
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
author_df = spark.createDataFrame(rows, ["Authors", "State"])
author_df.show()

+-------------+-----+
|      Authors|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



# DataFrameReader/DataFrameWriter 사용하기(p.60~)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Chapter03").getOrCreate()

# 스키마 정의
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                         StructField('UnitID', StringType(), True),
                         StructField('IncidentNumber', IntegerType(), True),
                         StructField('CallType', StringType(), True),
                         StructField('CallDate', StringType(), True),
                         StructField('WatchDate', StringType(), True),
                         StructField('CallFinalDisposition', StringType(), True),
                         StructField('AvailableDtTm', StringType(), True), 
                         StructField('Address', StringType(), True),
                         StructField('City', StringType(), True), 
                         StructField('Zipcode', StringType(), True), 
                         StructField('Battalion', StringType(), True),
                         StructField('StationArea', StringType(), True), 
                         StructField('Box', StringType(), True),
                         StructField('OriginalPriority', StringType(), True),
                         StructField('Priority', StringType(), True), 
                         StructField('FinalPriority', IntegerType(), True), 
                         StructField('ASLUnit', BooleanType(), True), 
                         StructField('CalltypeGroup', StringType(), True), 
                         StructField('NumAlarms', IntegerType(), True),
                         StructField('UnitType', StringType(), True), 
                         StructField('UnitSequenceInCallDistrict', IntegerType(), True), 
                         StructField('FirePreventionDistirct', StringType(), True),
                         StructField('SupervisiorDistrict', StringType(), True),
                         StructField('Neighborhood', StringType(), True),
                         StructField('Location', StringType(), True), 
                         StructField('RowID', StringType(), True),
                         StructField('Delay', FloatType(), True) 
                         ])

sf_fire_file = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/chapter3/data/sf-fire-calls.csv"
fire_df = spark.read.csv(sf_fire_file, header = True, schema = fire_schema)

In [4]:
fire_df.show(5, False)

+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+-------------------+---------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                    |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ASLUnit|CalltypeGroup|NumAlarms|UnitType|UnitSequenceInCallDistrict|FirePreventionDistirct|SupervisiorDistrict|Neighborhood         |Location                             |RowID        |Delay    |
+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+

## 데이터 프레임을 파케이 파일/ SQL 테이블로 저장하기

In [18]:
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [19]:
spark.version

'3.3.2'

In [5]:
parquet_path = "C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/chapter3/data/fire_df.parquet"
fire_df.write.format("parquet").mode('overwrite').save(parquet_path)

In [10]:
parquet_table = 'C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/chapter3/data/few_fire_tbl'
fire_df.write.format("parquet").mode('overwrite').saveAsTable(parquet_table)

ParseException: 
Syntax error at or near ':'(line 1, pos 1)

== SQL ==
C:/0. Modeling/1. Modeling Code/pyspark_dataset/LearningSparkV2-master/chapter3/data/few_fire_tbl
-^^^


## 프로젝션과 필터 (p.63~)

In [ ]:
few_fire_df = (fire_df
              .select("IncidentNumber", "AvailableDtTm","CallType")
              .where(col("CallType") != "Medical Incident"))

few_fire_df.show(5, truncate = False)

In [ ]:
### 신고로 기록된 Calltype 종류

from pyspark.sql.functions import *

(fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .agg(countDistinct("CallType").alias("DistinctCallTypes"))
    .show()
)

In [ ]:
### null이 아닌 신고 타입의 목록

(fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .distinct()
    .show(10, False)
)

## 칼럼 이름 변경

In [ ]:
new_fire_df = fire_df.withColumnRenamed("Delay","ResponseDelayedinMins")

(new_fire_df
    .select("ResponseDelayedinMins")
    .where(col("ResponseDelayedinMins")>5)
    .show(5, False)
)

## 칼럼 속성 변경

In [ ]:
fire_ts_df= (new_fire_df
             .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
             .drop("CallDate")
             .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
             .drop("WatchDate")
             .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a"))
             .drop("AvailableDtTm") 
            )

In [ ]:
fire_ts_df.show(5,False)

In [ ]:
(fire_ts_df
    .select("IncidentDate", "OnWatchDate","AvailableDtTS")
    .show(5, False))

In [ ]:
(fire_ts_df
    .select(year("IncidentDate"))
    .distinct()
    .orderBy(year("IncidentDate"))
    .show()
)

## 집계 연산

In [ ]:
(fire_ts_df
    .select("CallType")
    .where(col('CallType').isNotNull())
    .groupby("CallType")
    .count()
    .orderBy("count", ascending = False)
    .show(n=10, truncate = False)
)

In [ ]:
import pyspark.sql.functions as F

(fire_ts_df
    .select(F.sum("NumAlarms"), F.avg("ResponseDelayedinMins"),
           F.min("ResponseDelayedinMins"), F.max("ResponseDelayedinMins"))
    .show()
)

# 데이터세트 API

## 정적 타입 객체, 동적 타입 객체, 포괄적인 Row

In [ ]:
row = Row(350, True, "Learning Spark 2E", None)

In [ ]:
row[2]